# Efficiency Simulation

In [10]:
from __future__ import annotations
from dataclasses import dataclass
from coordinates import Location, Checkpoint, device
import math as m
import copy
from collections.abc import Callable

from torch import FloatTensor, IntTensor, tensor
import torch
import torch.autograd.profiler as profiler
from tqdm.auto import tqdm

In [11]:
import pandas as pd
from coordinates import Checkpoint, Location, device
from torch import tensor

track_data = pd.read_csv("./sem_2023_us.csv")

track_data = track_data.rename(columns={
    "Metres above sea level": "Altitude"
})

track_data.head(10)

checkpoints: list[Checkpoint] = []
for i, row in track_data.iterrows():
    location = Location.construct(row["Latitude"], row["Longitude"], row["Altitude"])
    print(f"Lat {row['Latitude']}, Long {row['Longitude']}, Alt {row['Altitude']}, x {location.x}, y {location.y}, z {location.z}")
    checkpoints.append(Checkpoint(location, location))

print(f"Found {len(checkpoints)} checkpoints")

Lat 39.799168014, Long -86.23801415599999, Alt 222.3313, x -59843.80593682127, y -35436.914632370695, z 3.3495037353515613
Lat 39.799173325, Long -86.23799871, Alt 222.3617, x -59864.98942870321, y -35466.98026063293, z 3.3799037353515757
Lat 39.799179425, Long -86.237984708, Alt 222.3676, x -59891.17269970616, y -35498.34457240533, z 3.3858037353515726
Lat 39.799186418, Long -86.23797202, Alt 222.3918, x -59922.87791080261, y -35531.41276884545, z 3.4100037353515518
Lat 39.799193315, Long -86.237959625, Alt 222.3986, x -59954.19794881111, y -35563.94105741847, z 3.41680373535155
Lat 39.799200062, Long -86.23794597999999, Alt 222.4268, x -59984.08717097528, y -35597.04310335964, z 3.4450037353515484
Lat 39.799207064, Long -86.237931887, Alt 222.4488, x -60015.1353241913, y -35631.34579178132, z 3.467003735351568
Lat 39.799214173, Long -86.237918039, Alt 222.46920000000003, x -60046.88617331674, y -35665.78211542405, z 3.4874037353515916
Lat 39.799221068, Long -86.237904511, Alt 222.482

In [12]:
from graph import Graph, Node, Transition 
def get_coefficient_of_drag(bearing: float) -> float:
    return 0.33

def get_projected_area(bearing: float) -> float:
    """Returns in mm^2"""
    return 943416

g = Graph.construct(
    checkpoints=checkpoints,
    n_points_per_checkpoint=1,
    max_velocity=42 * 1000 / 3600,  # Max velocity the car is allowed to go is 42 km/h
    velocity_step_size=4000 / 3600,
    wind_velocity=5000 / 3600,
    wind_bearing=200,
    mass=108000,
    coefficient_of_friction=0.03, # TODO: Figure this out
    get_coefficient_of_drag=get_coefficient_of_drag,
    get_projected_area=get_projected_area,
)

100%|██████████| 2695/2695 [02:49<00:00, 15.95it/s]


In [13]:
def topological_order(graph: Graph) -> list[int]:
    '''Returns the ids in topo order'''
    # Topological sorting using DFS
    topological_order: list[int] = []
    marked_nodes: set[int] = set() # Used to track visited nodes

    def visit(node: Node):
        if node.id in marked_nodes:
            return

        for transition in node.transitions:
            visit(transition.target)

        marked_nodes.add(node.id)
        topological_order.insert(0, node.id)

    # Generate the topological order
    for node_id in graph.nodes.keys():
        if node_id not in marked_nodes:
            visit(graph.nodes[node_id])

    return topological_order

def cheapest_path(graph: Graph, work_weight: float, time_weight: float):
    # Topological sorting using DFS
    sorted_nodes = topological_order(graph)

    # Get all node ids
    node_ids = list(graph.nodes.keys())

    # Initialize cost and time dictionaries
    min_cost  = {node_id: float('inf') for node_id in node_ids}
    min_energy  = {node_id: float('inf') for node_id in node_ids}
    min_time = {node_id: float('inf') for node_id in node_ids}
    prev_node = {node_id: None for node_id in node_ids}

    min_cost[graph.start.id] = 0
    min_energy[graph.start.id] = 0
    min_time[graph.start.id] = 0

    # Iterate through nodes in topological order
    for node_id in sorted_nodes:
        current_node = graph.nodes[node_id]

        for transition in current_node.transitions:
            neighbor_id = transition.target.id
            new_time = min_time[node_id] + transition.time_required
            new_energy = min_energy[node_id] + transition.work_required

            # Calculate the new weighted cost
            weighted_cost = work_weight * new_energy + time_weight * new_time

            # Check if the new path offers a lower cost
            if weighted_cost < min_cost[neighbor_id]:
                min_cost[neighbor_id] = weighted_cost
                min_time[neighbor_id] = new_time
                min_energy[neighbor_id] = new_energy
                prev_node[neighbor_id] = node_id

    # Find the minimum cost path to the end node
    if min_cost[graph.end.id] == float('inf'):
        raise ValueError("No path found within reasonable time.")
    
    # Reconstruct the path
    path = []
    node_id = graph.end.id
    while node_id is not None:
        path.append(node_id)
        node_id = prev_node[node_id]

    path.reverse()  # Reverse to get the correct order from start to end
    return path, min_cost[graph.end.id], min_time[graph.end.id], min_energy[graph.end.id]

In [14]:
energy_weight = 1

node_ids = []
work = 0
time = 0
energy = 0

node_ids, cost, time, energy = cheapest_path(g, energy_weight, 1)
print(f"Energy {energy/1000} kJ | Time {time / 60} minutes | Cost {cost}")


Energy 7147440.107658505 kJ | Time 5272.420291425109 minutes | Cost 7147756452.875991
